This is a simple Convolutional Neural Network (CNN) model to classify images of clock faces.

The original dataset is taken from
https://www.kaggle.com/datasets/gpiosenka/time-image-datasetclassification
which contains images of clock faces with different times.
The dataset has 144 classes (from 1:00, 1:05, 1:10, ..., 12:55).
In this example, we have reduced dataset to only 12 classes, each representing an exact hour on the clock (i.e. 1:00,
2:00, ..., 12:00). The reduced dataset is available here given in `dataset-12` folder.

The dataset is divided into three folders: `train`, `test` and `valid`. Each of these folders contains subfolders for each class. The images are stored in the subfolders. The images are in RGB and have a size of 224x224 pixels. For each class, there are 80 images in the `train` folder, 10 images in the `test` folder and 10 images in the `valid` folder.

The CNN architecture leverages convolutional layers to extract spatial features from the clock face images, pooling layers to reduce dimensionality, and fully connected layers to map these features to their corresponding classes. The model is trained using a categorical cross-entropy loss function and optimized with an algorithm such as Adam. Through multiple epochs of training, the CNN learns to recognize the unique patterns associated with each hour, ultimately achieving accurate classification of clock times in the dataset.

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.listdir('/content/drive/MyDrive/lesson')

import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

Mounted at /content/drive
TensorFlow version: 2.18.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from google.colab import drive

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import time

############################################################
# define dataset properties
# - image size is 224x224
# - number of classes is 12 (from 1:00 to 12:00)
############################################################

IMG = 224
num_of_classes = 12

############################################################
# set training parameters
############################################################

batch_size = 32
num_epochs = 15

############################################################
# define the model
############################################################

model = tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG, IMG, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(128, activation='relu'),

    tf.keras.layers.Dense(num_of_classes, activation='softmax')
])

print(model.summary())

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

############################################################
# load the dataset
############################################################

train_folder = "/content/drive/MyDrive/lesson/dataset-12/train" # training
test_folder = "/content/drive/MyDrive/lesson/dataset-12/valid"  # validation

## training set
print("Loading training set...")
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.2)

training_set = train_datagen.flow_from_directory(train_folder,
                                              target_size=(IMG, IMG),
                                              color_mode='grayscale',
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=True)

## the class labels are in the same order as the folders
## so expect to see class_label = {0: '01-00', 1: '02-00', ..., 11: '12-00'}
## which means that the predicted time is f"{predicted_class_index+1}:00"
test_datagen = ImageDataGenerator(rescale=1. / 255)

class_labels = training_set.class_indices
class_labels = dict((v, k) for k, v in class_labels.items())  # invert dict
print("Labels:")
print(class_labels)

test_set = test_datagen.flow_from_directory(test_folder,
                                             target_size=(IMG, IMG),
                                             color_mode='grayscale',
                                             batch_size=batch_size,
                                             class_mode='categorical',
                                             shuffle=False)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 43264)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 43264)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     5,537,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 12)             │         1,548 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,567,532 (21.24 MB)

 Trainable params: 5,567,532 (21.24 MB)

 Non-trainable params: 0 (0.00 B)

None
Loading training set...
Found 960 images belonging to 12 classes.
Labels:
{0: '01-00', 1: '02-00', 2: '03-00', 3: '04-00', 4: '05-00', 5: '06-00', 6: '07-00', 7: '08-00', 8: '09-00', 9: '10-00', 10: '11-00', 11: '12-00'}
Found 120 images belonging to 12 classes.


In [3]:
############################################################
# train the model
############################################################

best_model_file = '/content/drive/MyDrive/lesson/dataset-12/best_model.keras'
best_model = ModelCheckpoint(best_model_file,
                             monitor='val_accuracy',
                             save_best_only=True,
                             mode='max',
                             verbose=1)

## train starts
t0 = time.time()
history = model.fit(training_set,
                    steps_per_epoch=int(np.ceil(training_set.samples/batch_size)),
                    epochs=num_epochs,
                    validation_data=test_set,
                    validation_steps=int(np.ceil(test_set.samples/batch_size)),
                    callbacks=[best_model],
                    verbose=1)

## train ends
t1 = int(time.time() - t0)
print(f"Total train time: {str(t1)} seconds")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 16s/step - accuracy: 0.0883 - loss: 2.7854 
Epoch 1: val_accuracy improved from -inf to 0.16667, saving model to /content/drive/MyDrive/lesson/dataset-12/best_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 556s 18s/step - accuracy: 0.0879 - loss: 2.7795 - val_accuracy: 0.1667 - val_loss: 2.4844
Epoch 2/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - accuracy: 0.1082 - loss: 2.4872
Epoch 2: val_accuracy did not improve from 0.16667
30/30 ━━━━━━━━━━━━━━━━━━━━ 17s 332ms/step - accuracy: 0.1075 - loss: 2.4872 - val_accuracy: 0.0917 - val_loss: 2.4820
Epoch 3/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step - accuracy: 0.1392 - loss: 2.4784
Epoch 3: val_accuracy improved from 0.16667 to 0.23333, saving model to /content/drive/MyDrive/lesson/dataset-12/best_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 324ms/step - accuracy: 0.1391 - loss: 2.4783 - val_accuracy: 0.2333 - val_loss: 2.4076
Epoch 4/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step - accuracy: 0.1786 - loss: 2.4